# Ch10. 워드 임베딩 (Word Embedding)

# v04. 글로브 (GloVe)

## 4.5 GloVe 훈련시키기

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

'2.2.0-rc1'

<br>

### 4.5.1 GloVe 설치

- GloVe 패키지 설치

In [ ]:
!pip install glove_python

<br>

### 4.5.2 훈련 데이터

- GloVe의 입력이 되는 훈련 데이터는 '영어와 한국어 Word2Vec 학습하기' 챕터에서 사용한 영어 데이터를 재사용한다.
- 모든 동일한 전처리를 마치고 이전과 동일하게 `result`에 결과가 저장되어 있다고 가정한다.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

# 데이터 다운로드
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))

    # xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
    parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [7]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

content_text[:200]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: expl"

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행
sent_text = sent_tokenize(content_text)

sent_text[:2]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.']

In [10]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

normalized_text[:2]

['here are two reasons companies fail they only do more of the same or they only do what s new ',
 'to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation ']

In [ ]:
# 각 문장에 대해서 NLTK를 이용해 단어 토큰화 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [12]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [13]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


<br>

### 4.5.3 훈련

In [14]:
from glove import Corpus, Glove

corpus = Corpus()

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)

# 학습에 이용할 쓰레드의 개수 4 (no_threads=4)
# 에포크 20 (epochs=20)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


<br>

### 4.5.4 `glove.most_similar()`

- `glove.most_similar()`는 입력 단어와 가장 유사한 단어들의 리스트를 출력한다.

In [15]:
# "man"
model_result1 = glove.most_similar("man")
print(model_result1)

[('woman', 0.9656968877121709), ('guy', 0.8923934313809874), ('girl', 0.8749832695592937), ('young', 0.8376841373272359)]


In [16]:
# "boy"
model_result2 = glove.most_similar("boy")
print(model_result2)

[('girl', 0.9462837151558987), ('woman', 0.848592139278506), ('kid', 0.8288868910342776), ('man', 0.8287990390041283)]


In [17]:
# "university"
model_result3 = glove.most_similar("university")
print(model_result3)

[('harvard', 0.886769949567571), ('mit', 0.8491405049409272), ('stanford', 0.836196635356568), ('cambridge', 0.8324910126415979)]


In [18]:
# "water"
model_result4 = glove.most_similar("water")
print(model_result4)

[('clean', 0.8419920899161488), ('fresh', 0.8382582571650625), ('air', 0.8347967096095402), ('electricity', 0.8096120117700837)]


In [19]:
# "physics"
model_result5 = glove.most_similar("physics")
print(model_result5)

[('chemistry', 0.8898927495028395), ('economics', 0.881057540215464), ('beauty', 0.8733432303405607), ('mathematics', 0.866095510956594)]


In [20]:
# "muscle"
model_result6 = glove.most_similar("muscle")
print(model_result6)

[('nerve', 0.8369865000720708), ('tissue', 0.8288942483958321), ('channel', 0.7617123128191436), ('skeletal', 0.7602536387613167)]


In [21]:
# "clean"
model_result7 = glove.most_similar("clean")
print(model_result7)

[('water', 0.8419920899161488), ('fresh', 0.8416432408847341), ('heat', 0.8092694145045692), ('wind', 0.8022735591670345)]
